In [36]:
import spotipy
import json
import webbrowser
import urllib.request
import overpass
import spotipy.util as util
import geohash
import pygeohash as pgh

In [37]:
with open("geocode_keys.txt", "r") as geokey_file:
    geo_key = geokey_file.read()

In [38]:
address = "15 Elm St, Totowa, NJ, 07512, US"
nospace_address = (
    address
        .replace(",", "")
        .replace(" ", "+")
)

# print(nospace_address)

map_url = f"https://geocode.maps.co/search?q={nospace_address}&api_key={geo_key}"

map_request = urllib.request.Request(map_url)
map_response = urllib.request.urlopen(map_request)

map_data = json.loads(map_response.read())
lat = map_data[0]['lat']
lon = map_data[0]['lon']
print(lat, lon)

40.923232 -74.202799


In [39]:
test_hash = pgh.encode(40.923232, -74.202799, precision=5)
print(test_hash)

dr70z


In [40]:
with open("ticketmaster_keys.txt", "r") as key_file:
    ticketmaster_key = key_file.read()

In [41]:
geohash = "gcpuv"
unit = "km"
radius = "10"
genreID = "KZazBEonSMnZfZ7v6F1"
url = f"https://app.ticketmaster.com/discovery/v2/events.json?size=20&radius={radius}&geoPoint={test_hash}&classificationName=music&apikey={ticketmaster_key}"

tm_request = urllib.request.Request(url)
tm_response = urllib.request.urlopen(tm_request)

full_event_listings = json.loads(tm_response.read())
event_names = full_event_listings['_embedded']['events']
print(event_names)


[{'name': 'AccessMore Presents "Wonder: A Night of Story & Song Celebrating Christmas" with Christine Caine, Lisa Harper, and Special Guest Jon Reddick - Wayne, NJ', 'type': 'event', 'id': 'rZ7HnEZ1AK_4vV', 'test': False, 'url': 'https://www.ticketweb.com/event/accessmore-presents-wonder-a-night-calvary-temple-international-tickets/13655074', 'locale': 'en-us', 'images': [{'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/c/8cf/a6653880-7899-4f67-8067-1f95f4d158cf_124761_RETINA_LANDSCAPE_16_9.jpg', 'width': 1136, 'height': 639, 'fallback': True}, {'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/c/8cf/a6653880-7899-4f67-8067-1f95f4d158cf_124761_RETINA_PORTRAIT_16_9.jpg', 'width': 640, 'height': 360, 'fallback': True}, {'ratio': '1_1', 'url': 'https://i.ticketweb.com/i/00/12/43/35/34_Edp.jpg?v=6', 'width': 427, 'height': 427, 'fallback': False}, {'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/c/8cf/a6653880-7899-4f67-8067-1f95f4d158cf_124761_RECOMENDATION_16_9.jpg', 'width': 1

In [42]:
artist_names = [artist['name'] for artist in event_names]
artist_names_no_repeats = list(dict.fromkeys(artist_names))

print(artist_names_no_repeats)

['AccessMore Presents "Wonder: A Night of Story & Song Celebrating Christmas" with Christine Caine, Lisa Harper, and Special Guest Jon Reddick - Wayne, NJ', 'Dirty Blond (Blondie Tribute) , Heart Tribute "Dream of the Archer" ,', 'Issues (Korn Tribute), T.O.A.D. (System of a Down Tribute), Variant Se', 'Throne of Exile, Ocean of Illusions, Soul Debt, the Adversary, Maximus', 'Leftover Crack, Sadplant', 'Jersey Calling, Torment of Giants, Point Blank, Laveau, Humiliate', 'Nirvana Tribute "Lounge Act", Ice Cold Chili Peppers (RHCP Tribute) ,', 'Phenomenon, Hindsight, Satin', 'Warrior Soul, All Sinners, Iron Priest (Iron Maiden/Judas Priest Tribu', 'Return To Dust, Mainline', 'Spread Eagle, Streetlight Circus, the Blues Vultures, Hyngd', "Rock 'n' Radio (70s, 80s and 90s Tribute Band)", "Start From Something, Beginner's Luck", 'Liliac, Amber Wild, Mainline, Kobi Reese', 'Deadlands, Renesans, Vya , Sinful By Nature', 'Skinlab, Dead By Wednesday', 'Warrior Soul', 'Venom Inc.', 'Senses Fail 

In [43]:
cred = "spotify_keys.json"
with open(cred, "r") as spotify_key_file:
    api_tokens = json.load(spotify_key_file)

spotify_key_file.close()

client_id = api_tokens['client_id']
client_secret = api_tokens['client_secret']
redirectURI = api_tokens['redirect']
username = api_tokens['username']

scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri=redirectURI)

In [ ]:
sp = spotipy.Spotify(auth=token)

playlist_songs = []

for artist in artist_names_no_repeats:
    returned_song = sp.search(q=artist, type="track", limit=1)

    if returned_song['tracks']['items'][0]['uri'] not in playlist_songs:
        playlist_songs.append(returned_song['tracks']['items'][0]['uri'])

#print(playlist_songs)

empty_playlist = sp.user_playlist_create(user=username, name=f"What's on in {address.upper()}", public=True, description=f"songs from artists who have played live within a 10km radius of {address.upper()}")
full_playlist = sp.user_playlist_add_tracks(username, empty_playlist['id'], playlist_songs)

In [45]:
webbrowser.open(empty_playlist['external_urls']['spotify'])

True